In [10]:
# Carregando a API Key
from dotenv import load_dotenv
import os

# Carrega as variaveis do arquivo .env 
load_dotenv()

api_key = os.getenv("API_KEY")

if api_key:
    print("API Key carregada com sucesso!")
else:
    print("API Key não encontrada. Verifique o arquivo .env.")

API Key carregada com sucesso!


In [11]:
# Acessando a API no seu endpoint

import http.client

conn = http.client.HTTPSConnection("v1.american-football.api-sports.io")

headers = {
    'x-rapidapi-host': "v1.american-football.api-sports.io",
    'x-rapidapi-key': api_key
    }

conn.request("GET", "/standings?league=1&season=2024", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))



In [6]:
# Salvar os dados brutos em um arquivo JSON

import json

with open('data_bronze.json', 'w') as f:
    json.dump(data, f)


In [7]:
import pandas as pd

# Organizar os dados em um DataFrame
df = pd.DataFrame(data['response'])

# Salvar o DataFrame em um arquivo CSV (silver layer)
df.to_csv('data_silver.csv', index=False)

In [13]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd

# Carregar chave da API do arquivo .env
load_dotenv()
api_key = os.getenv("API_KEY")

# Função para consultar os dados da API para uma temporada específica
def fetch_nfl_data_for_season(season):
    url = f"https://v2.nfl.api-sports.io/standings?season={season}"

    headers = {
        'x-rapidapi-host': 'v2.nfl.api-sports.io',
        'x-rapidapi-key': api_key
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Função para processar e salvar dados de várias temporadas
def fetch_and_save_multiple_seasons(seasons, output_file):
    all_data = []
    
    for season in seasons:
        print(f"Buscando dados para a temporada {season}...")
        data = fetch_nfl_data_for_season(season)
        
        # Processar e adicionar os dados ao conjunto
        if 'response' in data:
            for record in data['response']:
                record['season'] = season  # Adicionar a temporada como uma nova coluna
                all_data.append(record)
    
    # Salvar todos os dados em um DataFrame
    df = pd.DataFrame(all_data)
    df.to_csv(output_file, index=False)
    print(f"Dados salvos em {output_file}")

# Especificar as temporadas desejadas
seasons = list(range(2022, 2025))  # Temporadas de 2015 a 2024
output_file = "nfl_data_multiple_seasons.csv"

# Buscar e salvar os dados
fetch_and_save_multiple_seasons(seasons, output_file)


Buscando dados para a temporada 2022...


ConnectionError: HTTPSConnectionPool(host='v2.nfl.api-sports.io', port=443): Max retries exceeded with url: /standings?season=2022 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002D3C94372F0>: Failed to resolve 'v2.nfl.api-sports.io' ([Errno 11001] getaddrinfo failed)"))